In [ ]:
from typing import Annotated, List, Optional, Dict, Literal

import os

from langchain_core.tools import tool

# from langgraph.prebuilt import create_agent 
from langchain.agents import create_agent 

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command

from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, trim_messages
from typing_extensions import TypedDict

In [ ]:
class State(MessagesState):
    next: str

def make_supervisor_node(llm: BaseChatModel, members: List[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers: {members}. Given the following user request,"
        " respond with the workers to act next. Each worker will perform a"
        " task and respond with their results and status. When finished."
        ""
        "When finished, respond with FINISH."
    )

    class Router(TypedDict):
        next: Literal[*options]
    
    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        messages = [
            {"role": "system", "content": system_prompt}
        ] + state["messages"]
    
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END
        return Command(goto=goto, update={"next": goto})
    
    return supervisor_node

In [ ]:
def search_node(state: State) -> Command[Literal["supervisor"]]:
    result = search_agent.invoke(state)
    return Command(
        update = {
            "messages": [HumanMessage(content=result["messages"][-1].content, name="search")]
        },
        goto = "supervisor"
    )

web_scrapper_agent = create_react_agent(llm, tools=[scrape_webpages])

def web_scrapper_node(state: State) -> Command[Literal["supervisor"]]:
    result = web_scrapper_agent.invoke(state)
    return Command(
        update = {
            "messages": [HumanMessage(content=result["messages"][-1].content, name="web_scrapper")]
        },
        goto = "supervisor"
    )

research_supervisor_node = make_supervisor_node(llm, ["search", "web_scrapper"])